In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
import shap

from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.datasets import AdultDataset, BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

from IPython.display import Markdown, display

%matplotlib inline


In [16]:
np.random.seed(1)

In [17]:
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]
# We want to difine a custom preprocesing function (custom_preprocessing(df)) from the standat dataset class 
# that will be used to transform the dataset

def custom_preprocessing(df):
    median_age = df['age'].median()
    df['age_binary'] = df['age'].apply(lambda x: 0 if x <= median_age else 1)
    df = df.drop('age', axis=1)
    df['race'] = df['race'].apply(lambda x: 1 if x =="White"  else 0)
    df['sex'] =df['sex'].apply(lambda x: 1 if x =="Male"  else 0)
    return df
# So what we did is to add a new column 'age_binary' to the dataset and drop the 'age' column, in order to 
# binarise the age column.
# Load the dataset with the library aif360
dataset= AdultDataset(custom_preprocessing=custom_preprocessing,
                          protected_attribute_names=['age_binary', 'sex'], # race will remain because in the original library is defined with this protecte attribute
                          privileged_classes=[np.array([1.0]),np.array([1.0]) ]) # We supposed that the privileged class is the old white male. It's also defined like this in the original library

dataset_orig_train, dataset_orig_vt = dataset.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)


IOError: [Errno 2] No such file or directory: 'C:\\Users\\kerim\\PycharmProjects\\pythonProject\\.venv\\Lib\\site-packages\\aif360\\datasets\\..\\data\\raw\\adult\\adult.data'
To use this class, please download the following files:

	https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
	https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test
	https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

and place them, as-is, in the folder:

	C:\Users\kerim\PycharmProjects\pythonProject\.venv\Lib\site-packages\aif360\data\raw\adult



SystemExit: 1

#### Clean up training data

In [18]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)

display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)

display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

NameError: name 'dataset_orig_train' is not defined

Step 3 Compute fairness metric on original training dataset
The fairness metric is Statistical Parity Difference whitch measures the disparity in positive outcomes between unprivileged and privileged groups. It compares the probability of receiving a positive outcome for members of the unprivileged group against that for members of the privileged group.

In [19]:
# Define the privileged and unprivileged groups in order to compute the disparate impact
privileged_groups = [{'age_binary': 1, 'sex': 1}]  # Old males
unprivileged_groups = [{'age_binary': 0, 'sex': 0}]  # Young females


As it can be seen from the privious cell, we conclude that there is a bias in this dataset because the statistical paity metric is not equal to zero. More specifficaly, in the unprivileged group we have 2% of peopele are suffering of unfairness.  


 Step 4 Mitigate bias by transforming the original dataset via technique to ensure the classifier is fair. Here we want to use the Pre-Processing method Reweighting for fairness. This method will simply assigns weights to samples to balance the representation of protected groups in the training process.

In [20]:
# Compute the fairness metric statistical parity measure, which is the difference in the mean prediction between the unprivileged and privileged groups.
# A negative value indicates less favorable outcomes for the unprivileged groups. in order to see if the dataset is biased
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

NameError: name 'dataset_orig_train' is not defined

In [21]:
print('The 50 first instance weights originally:')
dataset.instance_weights[:50]

The 50 first instance weights originally:


NameError: name 'dataset' is not defined

We can clearly see from the cell above that the reweight method of transforming the dataset into a fair one worked.


Step 5 Compute fairness metric on transformed dataset


In [22]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(dataset_orig_train)
dataset_transf_train = RW.transform(dataset_orig_train)

NameError: name 'dataset_orig_train' is not defined

In [23]:
print('The 50 first instance weights after reweighing:')
dataset_transf_train.instance_weights[:50]

The 50 first instance weights after reweighing:


NameError: name 'dataset_transf_train' is not defined

In [24]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())


NameError: name 'dataset_transf_train' is not defined

### Train classifier on original data


In [25]:
# Extract data for the original training set
X_train = dataset_orig_train.features
y_train = dataset_orig_train.labels.ravel()

X_valid = dataset_orig_valid.features
y_valid = dataset_orig_valid.labels.ravel()

X_test = dataset_orig_test.features
y_test = dataset_orig_test.labels.ravel()

# Extract data for the reweighted (fair) training set
X_train_transf = dataset_transf_train.features
y_train_transf = dataset_transf_train.labels.ravel()
w_train_transf = dataset_transf_train.instance_weights

# Prepare a scaler to normalize features (helpful for logistic regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

X_train_transf_scaled = scaler.fit_transform(X_train_transf)
X_valid_transf_scaled = scaler.transform(X_valid)  # validation set stays the same
X_test_transf_scaled = scaler.transform(X_test)

NameError: name 'dataset_orig_train' is not defined

In [26]:
# ===========================
# Model Without Fairness Mitigation
# ===========================
clf_orig = LogisticRegression(solver='lbfgs', max_iter=200)
clf_orig.fit(X_train_scaled, y_train)

y_pred_test_orig = clf_orig.predict(X_test_scaled)

# Evaluate performance
print("===== Original Classifier (No Fairness Mitigation) =====")
print("Accuracy on test set:", accuracy_score(y_test, y_pred_test_orig))
print(classification_report(y_test, y_pred_test_orig))

# Compute fairness metrics on test set
test_bld_orig = dataset_orig_test.copy(deepcopy=True)
test_bld_orig.labels = y_pred_test_orig.reshape(-1,1)

metric_test_orig = ClassificationMetric(dataset_orig_test,
                                        test_bld_orig,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)
print("Statistical parity difference (original):", metric_test_orig.statistical_parity_difference())
print("Equal opportunity difference (original):", metric_test_orig.equal_opportunity_difference())

NameError: name 'X_train_scaled' is not defined

In [27]:
# ===========================
# Model With Fairness Mitigation (Reweighted)
# ===========================
clf_transf = LogisticRegression(solver='lbfgs', max_iter=200)
# Important: use the instance weights when training on the reweighted dataset
clf_transf.fit(X_train_transf_scaled, y_train_transf, sample_weight=w_train_transf)

y_pred_test_transf = clf_transf.predict(X_test_transf_scaled)

print("\n===== Fairness Mitigated Classifier (Reweighted) =====")
print("Accuracy on test set:", accuracy_score(y_test, y_pred_test_transf))
print(classification_report(y_test, y_pred_test_transf))

# Compute fairness metrics on test set for the fairness mitigated classifier
test_bld_transf = dataset_orig_test.copy(deepcopy=True)
test_bld_transf.labels = y_pred_test_transf.reshape(-1,1)

metric_test_transf = ClassificationMetric(dataset_orig_test,
                                          test_bld_transf,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=privileged_groups)
print("Statistical parity difference (reweighted):", metric_test_transf.statistical_parity_difference())
print("Equal opportunity difference (reweighted):", metric_test_transf.equal_opportunity_difference())

NameError: name 'X_train_transf_scaled' is not defined

In [28]:







# ===========================
# Comparison & Conclusions
# ===========================
print("\nComparison of Results:")
print("Original vs Reweighted:")
print(" - Test Accuracy: {:.4f} vs {:.4f}".format(accuracy_score(y_test, y_pred_test_orig), accuracy_score(y_test, y_pred_test_transf)))
print(" - Statistical Parity Difference: {:.4f} vs {:.4f}".format(metric_test_orig.statistical_parity_difference(),
                                                                  metric_test_transf.statistical_parity_difference()))
print(" - Equal Opportunity Difference: {:.4f} vs {:.4f}".format(metric_test_orig.equal_opportunity_difference(),
                                                                 metric_test_transf.equal_opportunity_difference()))




Comparison of Results:
Original vs Reweighted:


NameError: name 'y_test' is not defined